In [21]:
import math

import matplotlib.pyplot as plt
import torch
from   torch import nn, optim
import torch.nn.functional as F
from   torch.utils.data import DataLoader, TensorDataset
from   torchvision import datasets, transforms

In [2]:
W = torch.randn(256, 4) / math.sqrt(256)
W.requires_grad_()

tensor([[-0.0262,  0.0244, -0.0047, -0.0516],
        [ 0.0111,  0.0395,  0.0363, -0.0739],
        [-0.0468,  0.0285, -0.0957, -0.0715],
        ...,
        [-0.0854, -0.0859, -0.0786, -0.0552],
        [ 0.0080,  0.0071,  0.0519, -0.0544],
        [-0.0024,  0.0033, -0.0582,  0.0629]], requires_grad=True)

In [3]:
b = torch.zeros(4, requires_grad=True)

In [4]:
loss_func = F.cross_entropy
# loss = loss_func(mod(X), y)

In [5]:
#opt = optim.SGD(mod.parameter(), lr=ETA)

In [6]:
'''
with torch.no_grad():
    for param in mod.parameters:
        param -= param.grad * ETA # grad desc step
    mod.zero_grad()
'''

# Equivalent to:
'''
opt.step()
opt.zero_grad()
'''

'\nopt.step()\nopt.zero_grad()\n'

In [7]:
'''
train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batc_size=BATCH)
'''

# now instead of manually iterating through data like:
'''
for i in range((n - 1) // bs + 1):
    X_batch = X_train[start:end]
    y_batch = y_train[start:end]
    preds = mod(X_batch)
'''

# ...can simplify to
'''
for X_batch, y_batch in train_dataloader:
    preds = mod(X_batch)
'''

'\nfor X_batch, y_batch in train_dataloader:\n    preds = mod(X_batch)\n'

In [8]:
points = torch.tensor([1., 4., 2., 1., 3., 5.])
points[0]

tensor(1.)

In [9]:
float(points[0])

1.0

In [10]:
points.shape

torch.Size([6])

In [11]:
points = torch.tensor([[1., 4.], [2., 1.], [3., 5.]])
points.storage()

 1.0
 4.0
 2.0
 1.0
 3.0
 5.0
[torch.FloatStorage of size 6]

In [12]:
points.shape, points.size()

(torch.Size([3, 2]), torch.Size([3, 2]))

In [13]:
points.storage_offset() # index of 1st elem

0

In [14]:
points[1].storage_offset()

2

In [15]:
points.stride() 

(2, 1)

In [16]:
points = torch.tensor([[1., 2.], [3., 4.]], dtype=torch.float32)

In [17]:
points = torch.tensor(
    [[1., 2.], [3., 4.]], dtype=torch.float32, device='cpu')

In [20]:
#points2 = points.to(device='cuda')
#points3 = points.to(device='cuda:0')

In [22]:
# CNN for MNIST
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.cn1 = nn.Conv2d(1, 16, 3, 1)
        self.cn2 = nn.Conv2d(16, 32, 3, 1)
        self.dp1 = nn.Dropout2d(0.1)
        self.dp2 = nn.Dropout2d(0.25)
        self.fc1 = nn.Linear(12 * 12 * 32, 64)
        self.fc2 = nn.Lenear(64, 10)
        
    def forward(self, x):
        x = self.cn1(x)
        x = F.relu(x)
        x = self.cn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dp1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp2(x)
        x = self.fc2(x)
        op = F.log_softmax(x, dim=1)
        return op